<h1 style="color: #1e3a8a; background-color: #dbeafe; padding: 12px; border-left: 5px solid #2563eb; margin: 20px 0; font-weight: bold;">Modèle de Production - Home Credit Scoring API</h1>

**Objectif** : Entraîner le modèle LightGBM final optimisé et sauvegarder tous les artefacts nécessaires pour la production.

**Artefacts générés** :
- `model.pkl` : Modèle LightGBM entraîné
- `label_encoders.pkl` : Dictionnaire des LabelEncoders
- `onehot_encoder.pkl` : OneHotEncoder
- `feature_names.pkl` : Liste des noms de features après encodage
- `metrics.json` : Métriques de référence du modèle
- `threshold.json` : Seuil de décision optimal

In [11]:
# Imports
import pandas as pd
import numpy as np
import pickle
import json
from pathlib import Path

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, 
    f1_score, confusion_matrix, roc_curve
)
from lightgbm import LGBMClassifier

# Configuration
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("Imports terminés")

Imports terminés


<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Chargement des Données</h2>

In [12]:
print("="*80)
print("CHARGEMENT DES DONNÉES")
print("="*80)
print()

# Chemin vers les données
data_path = Path('../data/app_train_models.csv')

if not data_path.exists():
    raise FileNotFoundError(f"Fichier non trouvé : {data_path}")

# Chargement
df = pd.read_csv(data_path)

print(f"Dataset chargé : {data_path.name}")
print(f"Forme : {df.shape}")
print(f"Colonnes : {df.shape[1]}")
print(f"Observations : {df.shape[0]:,}")
print()

# Distribution de la target
if 'TARGET' in df.columns:
    target_dist = df['TARGET'].value_counts(normalize=True)
    print("DISTRIBUTION DE LA TARGET:")
    print(f"  Classe 0 (pas de défaut): {target_dist[0]:.1%}")
    print(f"  Classe 1 (défaut): {target_dist[1]:.1%}")
    print(f"  Ratio déséquilibre: {target_dist[0]/target_dist[1]:.1f}:1")
    print()

# Vérification valeurs manquantes
missing_count = df.isnull().sum().sum()
print(f"Valeurs manquantes : {missing_count}")
print()

print("Chargement terminé")
print()

CHARGEMENT DES DONNÉES

Dataset chargé : app_train_models.csv
Forme : (307511, 646)
Colonnes : 646
Observations : 307,511

DISTRIBUTION DE LA TARGET:
  Classe 0 (pas de défaut): 91.9%
  Classe 1 (défaut): 8.1%
  Ratio déséquilibre: 11.4:1

Valeurs manquantes : 0

Chargement terminé



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Split Train/Validation Stratifié</h2>

In [13]:
print("="*80)
print("SPLIT TRAIN/VALIDATION STRATIFIÉ")
print("="*80)
print()

# Séparation features et target
X = df.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = df['TARGET']

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")
print()

# Split stratifié 80/20
test_size = 0.2
random_state = 42

X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=test_size, 
    random_state=random_state,
    stratify=y
)

print("RÉSULTATS DU SPLIT:")
print(f"  Train set: {X_train.shape[0]:,} échantillons ({(1-test_size)*100:.0f}%)")
print(f"  Validation set: {X_val.shape[0]:,} échantillons ({test_size*100:.0f}%)")
print()

# Vérification stratification
original_proportions = y.value_counts(normalize=True)
train_proportions = y_train.value_counts(normalize=True)
val_proportions = y_val.value_counts(normalize=True)

print("VÉRIFICATION STRATIFICATION:")
print(f"{'Set':<12} {'Classe 0':<10} {'Classe 1':<10} {'Ratio':<10}")
print("-" * 50)
print(f"{'Original':<12} {original_proportions[0]:<10.1%} {original_proportions[1]:<10.1%} {original_proportions[0]/original_proportions[1]:<10.1f}")
print(f"{'Train':<12} {train_proportions[0]:<10.1%} {train_proportions[1]:<10.1%} {train_proportions[0]/train_proportions[1]:<10.1f}")
print(f"{'Validation':<12} {val_proportions[0]:<10.1%} {val_proportions[1]:<10.1%} {val_proportions[0]/val_proportions[1]:<10.1f}")
print()

print("Split terminé")
print()

SPLIT TRAIN/VALIDATION STRATIFIÉ

Features (X): (307511, 644)
Target (y): (307511,)

RÉSULTATS DU SPLIT:
  Train set: 246,008 échantillons (80%)
  Validation set: 61,503 échantillons (20%)

VÉRIFICATION STRATIFICATION:
Set          Classe 0   Classe 1   Ratio     
--------------------------------------------------
Original     91.9%      8.1%       11.4      
Train        91.9%      8.1%       11.4      
Validation   91.9%      8.1%       11.4      

Split terminé



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Encodage des Variables Catégorielles</h2>

**Stratégie** :
- **≤2 catégories** → Label Encoding (0/1)
- **>2 catégories** → One-Hot Encoding
- **Fit sur train uniquement** → Transform sur train et validation

In [14]:
print("="*80)
print("ENCODAGE DES VARIABLES CATÉGORIELLES")
print("="*80)
print()

# Identifier les variables catégorielles
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
print(f"Variables catégorielles détectées: {len(categorical_cols)}")
print()

if len(categorical_cols) == 0:
    print("Aucune variable catégorielle - Datasets déjà numériques")
    X_train_encoded = X_train.copy()
    X_val_encoded = X_val.copy()
    label_encoders = {}
    onehot_encoders = {}
else:
    # Analyser les variables catégorielles
    print("ANALYSE DES VARIABLES CATÉGORIELLES:")
    cols_label_encoding = []
    cols_onehot_encoding = []

    for col in categorical_cols:
        n_categories = X_train[col].nunique()
        if n_categories <= 2:
            cols_label_encoding.append(col)
            strategy = "Label Encoding"
        else:
            cols_onehot_encoding.append(col)
            strategy = "One-Hot Encoding"
        print(f"  • {col}: {n_categories} catégories → {strategy}")

    print()
    print(f"Label Encoding (≤2 cat): {len(cols_label_encoding)}")
    print(f"One-Hot Encoding (>2 cat): {len(cols_onehot_encoding)}")
    print()

    # Copier les datasets
    X_train_encoded = X_train.copy()
    X_val_encoded = X_val.copy()

    # Dictionnaires pour les encodeurs
    label_encoders = {}
    onehot_encoders = {}

    # LABEL ENCODING
    if len(cols_label_encoding) > 0:
        print("APPLICATION LABEL ENCODING:")
        for col in cols_label_encoding:
            print(f"  Encodage: {col}")

            # Fit sur train uniquement
            le = LabelEncoder()
            le.fit(X_train_encoded[col])

            # Transform train
            X_train_encoded[col] = le.transform(X_train_encoded[col])

            # Transform validation (gestion catégories nouvelles)
            val_unknown = ~X_val_encoded[col].isin(le.classes_)
            if val_unknown.sum() > 0:
                print(f"    {val_unknown.sum()} catégories nouvelles en validation")
                most_frequent = X_train[col].mode()[0]
                X_val_encoded.loc[val_unknown, col] = most_frequent

            X_val_encoded[col] = le.transform(X_val_encoded[col])
            label_encoders[col] = le
        print()
    # ONE-HOT ENCODING
    if len(cols_onehot_encoding) > 0:
        print("APPLICATION ONE-HOT ENCODING:")
        for col in cols_onehot_encoding:
            print(f"  Encodage: {col}")

            # Fit sur train uniquement
            ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            ohe.fit(X_train_encoded[[col]])

            # Noms des nouvelles colonnes
            feature_names_temp = [f"{col}_{cat}" for cat in ohe.categories_[0]]

            # Transform train
            train_encoded = ohe.transform(X_train_encoded[[col]])
            train_ohe_df = pd.DataFrame(train_encoded, columns=feature_names_temp, index=X_train_encoded.index)
            X_train_encoded = X_train_encoded.drop(columns=[col])
            X_train_encoded = pd.concat([X_train_encoded, train_ohe_df], axis=1)

            # Transform validation
            val_encoded = ohe.transform(X_val_encoded[[col]])
            val_ohe_df = pd.DataFrame(val_encoded, columns=feature_names_temp, index=X_val_encoded.index)
            X_val_encoded = X_val_encoded.drop(columns=[col])
            X_val_encoded = pd.concat([X_val_encoded, val_ohe_df], axis=1)

            onehot_encoders[col] = ohe
        print()

    print("ENCODAGE TERMINÉ:")
    print(f"  • Label Encoders créés: {len(label_encoders)}")
    print(f"  • One-Hot Encoders créés: {len(onehot_encoders)}")
    print()

# Vérification finale
remaining_objects_train = X_train_encoded.select_dtypes(include=['object']).columns.tolist()
remaining_objects_val = X_val_encoded.select_dtypes(include=['object']).columns.tolist()

print("VÉRIFICATION POST-ENCODAGE:")
print(f"  Variables 'object' restantes:")
print(f"    Train: {len(remaining_objects_train)}")
print(f"    Validation: {len(remaining_objects_val)}")

if len(remaining_objects_train) == 0 and len(remaining_objects_val) == 0:
    print("  ENCODAGE COMPLET - Tous datasets 100% numériques")
else:
    print(f"  ATTENTION - Variables non encodées détectées")

print()
print("Datasets finaux après encodage:")
print(f"  Train: {X_train_encoded.shape}")
print(f"  Validation: {X_val_encoded.shape}")
print()

# Nettoyer les noms de colonnes pour LightGBM
print("Nettoyage des noms de colonnes pour LightGBM...")
X_train_encoded.columns = X_train_encoded.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
X_val_encoded.columns = X_val_encoded.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)
print("Nettoyage terminé")
print()

# Sauvegarder les noms de features
feature_names = X_train_encoded.columns.tolist()
print(f"Noms de features sauvegardés : {len(feature_names)} features")
print()

ENCODAGE DES VARIABLES CATÉGORIELLES

Variables catégorielles détectées: 37

ANALYSE DES VARIABLES CATÉGORIELLES:
  • NAME_CONTRACT_TYPE: 2 catégories → Label Encoding
  • CODE_GENDER: 3 catégories → One-Hot Encoding
  • FLAG_OWN_CAR: 2 catégories → Label Encoding
  • FLAG_OWN_REALTY: 2 catégories → Label Encoding
  • NAME_TYPE_SUITE: 7 catégories → One-Hot Encoding
  • NAME_INCOME_TYPE: 8 catégories → One-Hot Encoding
  • NAME_EDUCATION_TYPE: 5 catégories → One-Hot Encoding
  • NAME_FAMILY_STATUS: 6 catégories → One-Hot Encoding
  • NAME_HOUSING_TYPE: 6 catégories → One-Hot Encoding
  • OCCUPATION_TYPE: 18 catégories → One-Hot Encoding
  • WEEKDAY_APPR_PROCESS_START: 7 catégories → One-Hot Encoding
  • ORGANIZATION_TYPE: 58 catégories → One-Hot Encoding
  • FONDKAPREMONT_MODE: 4 catégories → One-Hot Encoding
  • HOUSETYPE_MODE: 3 catégories → One-Hot Encoding
  • WALLSMATERIAL_MODE: 7 catégories → One-Hot Encoding
  • EMERGENCYSTATE_MODE: 2 catégories → Label Encoding
  • BUREAU_CREDI

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Entraînement du Modèle LightGBM Optimisé</h2>

**Hyperparamètres optimaux** (trouvés via Optuna - Trial 41) :
- `n_estimators` : 300
- `max_depth` : 7
- `learning_rate` : 0.0583
- `num_leaves` : 40
- `min_child_samples` : 30
- `subsample` : 0.7
- `colsample_bytree` : 0.7
- `reg_alpha` : 0.9
- `reg_lambda` : 0.8

**Coût métier baseline** : 30,118

In [15]:
print("="*80)
print("ENTRAÎNEMENT DU MODÈLE LIGHTGBM OPTIMISÉ")
print("="*80)
print()

# Hyperparamètres optimaux (du meilleur trial Optuna)
best_params = {
    'n_estimators': 300,
    'max_depth': 7,
    'learning_rate': 0.058264831213179456,
    'num_leaves': 40,
    'min_child_samples': 30,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'reg_alpha': 0.9,
    'reg_lambda': 0.8,
    'class_weight': 'balanced',
    'random_state': 42,
    'verbose': -1
}

print("HYPERPARAMÈTRES:")
for param, value in best_params.items():
    if param != 'verbose':
        print(f"  • {param:<20} : {value}")
print()

# Créer le modèle
print("Création du modèle LightGBM...")
model = LGBMClassifier(**best_params)
print("Modèle créé")
print()

# Entraînement
print(f"Entraînement sur {X_train_encoded.shape[0]:,} échantillons...")
import time
start_time = time.time()

model.fit(X_train_encoded, y_train)

train_time = time.time() - start_time
print(f"Entraînement terminé en {train_time:.2f}s")
print()

print("Modèle entraîné avec succès")
print()

ENTRAÎNEMENT DU MODÈLE LIGHTGBM OPTIMISÉ

HYPERPARAMÈTRES:
  • n_estimators         : 300
  • max_depth            : 7
  • learning_rate        : 0.058264831213179456
  • num_leaves           : 40
  • min_child_samples    : 30
  • subsample            : 0.7
  • colsample_bytree     : 0.7
  • reg_alpha            : 0.9
  • reg_lambda           : 0.8
  • class_weight         : balanced
  • random_state         : 42

Création du modèle LightGBM...
Modèle créé

Entraînement sur 246,008 échantillons...
Entraînement terminé en 18.26s

Modèle entraîné avec succès



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Évaluation sur Validation Set</h2>

In [16]:
print("="*80)
print("ÉVALUATION SUR VALIDATION SET")
print("="*80)
print()

# Prédictions
print("Génération des prédictions...")
y_val_proba = model.predict_proba(X_val_encoded)[:, 1]
y_val_pred = model.predict(X_val_encoded)
print("Prédictions générées")
print()

# Métriques (seuil par défaut 0.5)
print("MÉTRIQUES VALIDATION (Seuil 0.5):")
auc_roc = roc_auc_score(y_val, y_val_proba)
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f"  • AUC-ROC   : {auc_roc:.4f}")
print(f"  • Accuracy  : {accuracy:.4f}")
print(f"  • Precision : {precision:.4f}")
print(f"  • Recall    : {recall:.4f}")
print(f"  • F1-Score  : {f1:.4f}")
print()

# Matrice de confusion
cm = confusion_matrix(y_val, y_val_pred)
tn, fp, fn, tp = cm.ravel()

print("MATRICE DE CONFUSION:")
print(f"  • TN (Vrai Négatif)  : {tn:,}")
print(f"  • FP (Faux Positif)  : {fp:,}")
print(f"  • FN (Faux Négatif)  : {fn:,}")
print(f"  • TP (Vrai Positif)  : {tp:,}")
print()

# Coût métier (seuil 0.5)
cost_fn = fn * 10  # Faux négatifs coûtent 10x
cost_fp = fp * 1   # Faux positifs coûtent 1x
total_cost = cost_fn + cost_fp

print("COÛT MÉTIER (Seuil 0.5):")
print(f"  • Coût FN (défauts manqués)      : {cost_fn:,} ({fn:,} x 10)")
print(f"  • Coût FP (bons clients refusés) : {cost_fp:,} ({fp:,} x 1)")
print(f"  • COÛT TOTAL                     : {total_cost:,}")
print()

print("Évaluation terminée")
print()

ÉVALUATION SUR VALIDATION SET

Génération des prédictions...
Prédictions générées

MÉTRIQUES VALIDATION (Seuil 0.5):
  • AUC-ROC   : 0.7826
  • Accuracy  : 0.7492
  • Precision : 0.1931
  • Recall    : 0.6628
  • F1-Score  : 0.2991

MATRICE DE CONFUSION:
  • TN (Vrai Négatif)  : 42,788
  • FP (Faux Positif)  : 13,750
  • FN (Faux Négatif)  : 1,674
  • TP (Vrai Positif)  : 3,291

COÛT MÉTIER (Seuil 0.5):
  • Coût FN (défauts manqués)      : 16,740 (1,674 x 10)
  • Coût FP (bons clients refusés) : 13,750 (13,750 x 1)
  • COÛT TOTAL                     : 30,490

Évaluation terminée



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Optimisation du Seuil de Décision</h2>

**Objectif** : Trouver le seuil qui minimise le coût métier total

**Coût métier** : `Coût = (10 × FN) + (1 × FP)`

In [17]:
print("="*80)
print("OPTIMISATION DU SEUIL DE DÉCISION")
print("="*80)
print()

# Tester différents seuils
thresholds_to_test = np.linspace(0.1, 0.9, 90)
costs = []
recalls = []
precisions = []

print(f"Test de {len(thresholds_to_test)} seuils...")

for threshold in thresholds_to_test:
    # Prédictions avec ce seuil
    y_pred_threshold = (y_val_proba >= threshold).astype(int)

    # Matrice de confusion
    cm_t = confusion_matrix(y_val, y_pred_threshold)
    tn_t, fp_t, fn_t, tp_t = cm_t.ravel()

    # Coût métier
    cost = (fn_t * 10) + (fp_t * 1)
    costs.append(cost)

    # Métriques
    recall_t = recall_score(y_val, y_pred_threshold)
    precision_t = precision_score(y_val, y_pred_threshold)
    recalls.append(recall_t)
    precisions.append(precision_t)

# Trouver le seuil optimal
optimal_idx = np.argmin(costs)
optimal_threshold = thresholds_to_test[optimal_idx]
optimal_cost = costs[optimal_idx]
optimal_recall = recalls[optimal_idx]
optimal_precision = precisions[optimal_idx]

print(f"Seuil optimal trouvé : {optimal_threshold:.4f}")
print()

# Métriques au seuil optimal
y_val_pred_optimal = (y_val_proba >= optimal_threshold).astype(int)
cm_optimal = confusion_matrix(y_val, y_val_pred_optimal)
tn_opt, fp_opt, fn_opt, tp_opt = cm_optimal.ravel()

print("RÉSULTATS AU SEUIL OPTIMAL:")
print(f"  • Seuil optimal          : {optimal_threshold:.4f}")
print(f"  • Coût optimal           : {optimal_cost:,}")
print(f"  • Coût au seuil 0.5      : {total_cost:,}")
print(f"  • Économie réalisée      : {total_cost - optimal_cost:,} ({((total_cost - optimal_cost)/total_cost)*100:.1f}%)")
print()

print("MÉTRIQUES AU SEUIL OPTIMAL:")
print(f"  • Recall    : {optimal_recall:.4f}")
print(f"  • Precision : {optimal_precision:.4f}")
print(f"  • F1-Score  : {2 * (optimal_precision * optimal_recall) / (optimal_precision + optimal_recall):.4f}")
print()

print("MATRICE DE CONFUSION (Seuil optimal):")
print(f"  • TN (Vrai Négatif)  : {tn_opt:,}")
print(f"  • FP (Faux Positif)  : {fp_opt:,}")
print(f"  • FN (Faux Négatif)  : {fn_opt:,}")
print(f"  • TP (Vrai Positif)  : {tp_opt:,}")
print()

print("Optimisation du seuil terminée")
print()

OPTIMISATION DU SEUIL DE DÉCISION

Test de 90 seuils...
Seuil optimal trouvé : 0.5225

RÉSULTATS AU SEUIL OPTIMAL:
  • Seuil optimal          : 0.5225
  • Coût optimal           : 30,437
  • Coût au seuil 0.5      : 30,490
  • Économie réalisée      : 53 (0.2%)

MÉTRIQUES AU SEUIL OPTIMAL:
  • Recall    : 0.6389
  • Precision : 0.2023
  • F1-Score  : 0.3073

MATRICE DE CONFUSION (Seuil optimal):
  • TN (Vrai Négatif)  : 44,031
  • FP (Faux Positif)  : 12,507
  • FN (Faux Négatif)  : 1,793
  • TP (Vrai Positif)  : 3,172

Optimisation du seuil terminée



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Sauvegarde des Artefacts de Production</h2>

**Artefacts sauvegardés dans `../models/`** :
1. `model.pkl` - Modèle LightGBM entraîné
2. `label_encoders.pkl` - Dictionnaire des LabelEncoders
3. `onehot_encoder.pkl` - OneHotEncoder (dict)
4. `feature_names.pkl` - Liste des noms de features
5. `metrics.json` - Métriques de référence
6. `threshold.json` - Seuil de décision optimal

In [18]:
print("="*80)
print("SAUVEGARDE DES ARTEFACTS DE PRODUCTION")
print("="*80)
print()

# Créer le dossier models s'il n'existe pas
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)
print(f"Dossier de destination : {models_dir.resolve()}")
print()

# 1. Sauvegarder le modèle
print("1. Sauvegarde du modèle LightGBM...")
model_path = models_dir / 'model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"   ✓ Modèle sauvegardé : {model_path.name}")
print()

# 2. Sauvegarder les label encoders
print("2. Sauvegarde des Label Encoders...")
label_encoders_path = models_dir / 'label_encoders.pkl'
with open(label_encoders_path, 'wb') as f:
    pickle.dump(label_encoders, f)
print(f"   ✓ Label Encoders sauvegardés : {label_encoders_path.name}")
print(f"   • Nombre d'encodeurs : {len(label_encoders)}")
print()

# 3. Sauvegarder les one-hot encoders
print("3. Sauvegarde des One-Hot Encoders...")
onehot_encoders_path = models_dir / 'onehot_encoder.pkl'
with open(onehot_encoders_path, 'wb') as f:
    pickle.dump(onehot_encoders, f)
print(f"   ✓ One-Hot Encoders sauvegardés : {onehot_encoders_path.name}")
print(f"   • Nombre d'encodeurs : {len(onehot_encoders)}")
print()

# 4. Sauvegarder les noms de features
print("4. Sauvegarde des noms de features...")
feature_names_path = models_dir / 'feature_names.pkl'
with open(feature_names_path, 'wb') as f:
    pickle.dump(feature_names, f)
print(f"   ✓ Noms de features sauvegardés : {feature_names_path.name}")
print(f"   • Nombre de features : {len(feature_names)}")
print()

# 5. Sauvegarder les métriques de référence
print("5. Sauvegarde des métriques de référence...")
metrics = {
    'auc_roc': float(auc_roc),
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'f1_score': float(f1),
    'threshold_default': 0.5,
    'confusion_matrix': {
        'tn': int(tn),
        'fp': int(fp),
        'fn': int(fn),
        'tp': int(tp)
    },
    'business_cost': {
        'cost_fn': int(cost_fn),
        'cost_fp': int(cost_fp),
        'total_cost': int(total_cost)
    },
    'optimal_threshold': {
        'threshold': float(optimal_threshold),
        'recall': float(optimal_recall),
        'precision': float(optimal_precision),
        'total_cost': int(optimal_cost),
        'savings': int(total_cost - optimal_cost),
        'confusion_matrix': {
            'tn': int(tn_opt),
            'fp': int(fp_opt),
            'fn': int(fn_opt),
            'tp': int(tp_opt)
        }
    },
    'training_info': {
        'train_samples': int(X_train_encoded.shape[0]),
        'val_samples': int(X_val_encoded.shape[0]),
        'n_features': int(len(feature_names)),
        'train_time_seconds': float(train_time)
    }
}

metrics_path = models_dir / 'metrics.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"   ✓ Métriques sauvegardées : {metrics_path.name}")
print()

# 6. Sauvegarder le seuil optimal
print("6. Sauvegarde du seuil optimal...")
threshold_data = {
    'optimal_threshold': float(optimal_threshold),
    'default_threshold': 0.5,
    'description': 'Seuil de décision optimisé pour minimiser le coût métier (10×FN + 1×FP)'
}

threshold_path = models_dir / 'threshold.json'
with open(threshold_path, 'w') as f:
    json.dump(threshold_data, f, indent=2)
print(f"   ✓ Seuil optimal sauvegardé : {threshold_path.name}")
print()

print("="*80)
print("TOUS LES ARTEFACTS ONT ÉTÉ SAUVEGARDÉS AVEC SUCCÈS")
print("="*80)
print()

# Résumé des fichiers créés
print("FICHIERS CRÉÉS:")
for file in sorted(models_dir.iterdir()):
    if file.is_file():
        size = file.stat().st_size
        if size < 1024:
            size_str = f"{size} B"
        elif size < 1024*1024:
            size_str = f"{size/1024:.1f} KB"
        else:
            size_str = f"{size/(1024*1024):.1f} MB"
        print(f"  • {file.name:<25} : {size_str}")
print()

print("Le modèle est prêt pour la production !")
print()

SAUVEGARDE DES ARTEFACTS DE PRODUCTION

Dossier de destination : /Users/mounirmeknaci/Desktop/Data_Projects/Projet8/models

1. Sauvegarde du modèle LightGBM...
   ✓ Modèle sauvegardé : model.pkl

2. Sauvegarde des Label Encoders...
   ✓ Label Encoders sauvegardés : label_encoders.pkl
   • Nombre d'encodeurs : 5

3. Sauvegarde des One-Hot Encoders...
   ✓ One-Hot Encoders sauvegardés : onehot_encoder.pkl
   • Nombre d'encodeurs : 32

4. Sauvegarde des noms de features...
   ✓ Noms de features sauvegardés : feature_names.pkl
   • Nombre de features : 911

5. Sauvegarde des métriques de référence...
   ✓ Métriques sauvegardées : metrics.json

6. Sauvegarde du seuil optimal...
   ✓ Seuil optimal sauvegardé : threshold.json

TOUS LES ARTEFACTS ONT ÉTÉ SAUVEGARDÉS AVEC SUCCÈS

FICHIERS CRÉÉS:
  • .gitkeep                  : 0 B
  • feature_names.pkl         : 29.2 KB
  • label_encoders.pkl        : 615 B
  • metrics.json              : 824 B
  • model.pkl                 : 1.4 MB
  • onehot_

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Résumé Final</h2>

In [19]:
print("="*80)
print("RÉSUMÉ FINAL - MODÈLE DE PRODUCTION")
print("="*80)
print()

print("MODÈLE:")
print(f"  • Algorithme           : LightGBM")
print(f"  • N° estimators        : {best_params['n_estimators']}")
print(f"  • Max depth            : {best_params['max_depth']}")
print(f"  • Learning rate        : {best_params['learning_rate']:.4f}")
print()

print("DONNÉES:")
print(f"  • Dataset source       : app_train_models.csv")
print(f"  • Observations totales : {df.shape[0]:,}")
print(f"  • Features finales     : {len(feature_names)}")
print(f"  • Train samples        : {X_train_encoded.shape[0]:,}")
print(f"  • Validation samples   : {X_val_encoded.shape[0]:,}")
print()

print("PERFORMANCES (Validation Set):")
print(f"  • AUC-ROC              : {auc_roc:.4f}")
print(f"  • Recall               : {recall:.4f}")
print(f"  • Precision            : {precision:.4f}")
print(f"  • F1-Score             : {f1:.4f}")
print()

print("COÛT MÉTIER:")
print(f"  • Seuil par défaut (0.5)")
print(f"    - Coût total         : {total_cost:,}")
print(f"  • Seuil optimal ({optimal_threshold:.4f})")
print(f"    - Coût total         : {optimal_cost:,}")
print(f"    - Économie           : {total_cost - optimal_cost:,} ({((total_cost - optimal_cost)/total_cost)*100:.1f}%)")
print()

print("ARTEFACTS SAUVEGARDÉS:")
print(f"  • Emplacement          : {models_dir.resolve()}")
print(f"  • Nombre de fichiers   : 6")
print(f"    - model.pkl")
print(f"    - label_encoders.pkl")
print(f"    - onehot_encoder.pkl")
print(f"    - feature_names.pkl")
print(f"    - metrics.json")
print(f"    - threshold.json")
print()

print("="*80)
print("NOTEBOOK TERMINÉ - MODÈLE PRÊT POUR LA PRODUCTION")
print("="*80)

RÉSUMÉ FINAL - MODÈLE DE PRODUCTION

MODÈLE:
  • Algorithme           : LightGBM
  • N° estimators        : 300
  • Max depth            : 7
  • Learning rate        : 0.0583

DONNÉES:
  • Dataset source       : app_train_models.csv
  • Observations totales : 307,511
  • Features finales     : 911
  • Train samples        : 246,008
  • Validation samples   : 61,503

PERFORMANCES (Validation Set):
  • AUC-ROC              : 0.7826
  • Recall               : 0.6628
  • Precision            : 0.1931
  • F1-Score             : 0.2991

COÛT MÉTIER:
  • Seuil par défaut (0.5)
    - Coût total         : 30,490
  • Seuil optimal (0.5225)
    - Coût total         : 30,437
    - Économie           : 53 (0.2%)

ARTEFACTS SAUVEGARDÉS:
  • Emplacement          : /Users/mounirmeknaci/Desktop/Data_Projects/Projet8/models
  • Nombre de fichiers   : 6
    - model.pkl
    - label_encoders.pkl
    - onehot_encoder.pkl
    - feature_names.pkl
    - metrics.json
    - threshold.json

NOTEBOOK TERMINÉ - MOD